In [7]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from utils import label_accuracy_score, add_hist
import cv2

import numpy as np
import pandas as pd
from tqdm import tqdm
import ttach as tta

# 전처리를 위한 라이브러리
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

##내가 사용할 모델정보 py파일 import해야된다 + 데이터로더도 같이
from smp_unet import *
from dataloader import *
# GPU 사용 가능 여부에 따라 device 정보 저장
device = "cuda" if torch.cuda.is_available() else "cpu"

In [8]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_path = "/opt/ml/segmentation/input/data/test.json"

test_transform = A.Compose([
#                             A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.2, p=0.5),
                           ToTensorV2() 
])


test_dataset = CustomDataLoader(data_dir=test_path, mode='test', transform=test_transform)
test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        num_workers=4,
        shuffle=False,
        collate_fn=collate_fn,
    )

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [9]:
model_path = './test/pseudo_2/best.pth'
checkpoint = torch.load(model_path, map_location=device)
model = myModel("UnetPlusPlus","timm-efficientnet-b4")
model = model.to(device)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [10]:
tta_transform = tta.Compose([
    tta.HorizontalFlip(),
    tta.VerticalFlip(),
    tta.Rotate90([0,90])
])
tta_model = tta.SegmentationTTAWrapper(model, tta_transform, merge_mode = 'mean')

In [11]:
def test(model, data_loader, device):
    #### 슈도할때는 512
    #### 그냥 라벨링 할때는 256
    size = 256
    transform = A.Compose([A.Resize(size, size)])
    print('Start prediction.')
    model.eval()
    
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    
    with torch.no_grad():
        for step, (imgs, image_infos) in enumerate(tqdm(test_loader)):
            
            # inference (512 x 512)
            outs = model(torch.stack(imgs).to(device))
            oms = torch.argmax(outs.squeeze(), dim=1).detach().cpu().numpy()
            
            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)
                
            oms = np.array(temp_mask)
            
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)
            preds_array = np.vstack((preds_array, oms))
            
            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    
    return file_names, preds_array

In [12]:
# sample_submisson.csv 열기
submission = pd.read_csv('./sample_submission.csv', index_col=None)

# test set에 대한 prediction
file_names, preds = test(tta_model, test_loader, device)

##### image name을 npy로 저장
#### 슈도 라벨링할때만 사용
'''
name_list = []
for i in file_names:
    name_list.append(i)
np.save('/opt/ml/segmentation/input/data/img_name', name_list)
'''
# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)
    
    

# submission.csv로 저장
submission.to_csv("./submission11_deeplab.csv", index=False)

  0%|          | 0/103 [00:00<?, ?it/s]

Start prediction.


100%|██████████| 103/103 [08:13<00:00,  4.79s/it]


End prediction.


In [ ]:
##### 재현님 코드 참고했습니다!

In [8]:
df = pd.read_csv('/opt/ml/segmentation/unet3/submission10_deeplab.csv')

for i in range(len(df)):
    cur = df.iloc[i, :]
    arr = np.array(cur['PredictionString'].split()).reshape(512, 512)
    np.save('/opt/ml/segmentation/input/data/mask3_deeplab/' + format(i, '03') , arr)

In [32]:
print(len(arr[0]),len(arr[220]),len(arr[511]))

512 512 512
